In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [4]:
train = pd.read_csv('train.csv')


In [5]:
train.head()

GameId          PlayId  Team      X      Y     S     A   Dis  \
0  2017090700  20170907000118  away  73.91  34.84  1.69  1.13  0.40   
1  2017090700  20170907000118  away  74.67  32.64  0.42  1.35  0.01   
2  2017090700  20170907000118  away  74.00  33.20  1.22  0.59  0.31   
3  2017090700  20170907000118  away  71.46  27.70  0.42  0.54  0.02   
4  2017090700  20170907000118  away  69.32  35.42  1.82  2.43  0.16   

   Orientation     Dir  ...  Week           Stadium        Location  \
0        81.99  177.18  ...     1  Gillette Stadium  Foxborough, MA   
1        27.61  198.70  ...     1  Gillette Stadium  Foxborough, MA   
2         3.01  202.73  ...     1  Gillette Stadium  Foxborough, MA   
3       359.77  105.64  ...     1  Gillette Stadium  Foxborough, MA   
4        12.63  164.31  ...     1  Gillette Stadium  Foxborough, MA   

   StadiumType        Turf     GameWeather Temperature Humidity  WindSpeed  \
0      Outdoor  Field Turf  Clear and warm        63.0     77.0          8   
1      Outdoor  Field Turf  Clear and warm        63.0     77.0          8   
2      Outdoor  Field Turf  Clear and warm        63.0     77.0          8   
3      Outdoor  Field Turf  Clear and warm        63.0     77.0          8   
4      Outdoor  Field Turf  Clear and warm        63.0     77.0          8   

   WindDirection  
0             SW  
1             SW  
2             SW  
3             SW  
4             SW  

[5 rows x 49 columns]

In [8]:
x = train.drop(['Yards'], axis=1).copy()
y = train['Yards'].copy()

In [ ]:
#TODO
# clean up missing data
# preprocess features and standardize them
# encode categorical features
#https://www.kaggle.com/shahules/how-about-some-nn-keras-starter

In [22]:
#fix windspeed
def windspeed(x):
    x=str(x)
    if x.isdigit():
        return int(x)
    elif (x.isalpha()):
        return 0
    elif (x.isalnum()):
        return int(x.upper().split('M')[0])                             #return 12 incase of 12mp or 12 MPH
    elif '-' in x:
        return int((int(x.split('-')[0])+int(x.split('-')[1]))/2)   # return average windspeed incase of 11 - 20 etc..
    else:
        return 0
train['WindSpeed']=train['WindSpeed'].apply(windspeed)
train['WindSpeed'].fillna(train['WindSpeed'].mean(),inplace=True)

In [30]:
train['Temperature'].isna().sum()

48532

In [35]:
# interpolate missing data
train['Humidity'].astype('float32').fillna(method='ffill', inplace=True)
train['Temperature'].astype('float32').fillna(0, inplace=True)





In [58]:
print(np.nanmean(train['Orientation'].values))
na_map = {
    'Orientation': train['Orientation'].mean(),
    'Dir': train['Dir'].mean(),
    'DefendersInTheBox': (train['DefendersInTheBox'].mean()),
    'OffenseFormation': 'UNKNOWN'
}

train.fillna(na_map, inplace=True)
train['DefendersInTheBox'].isna().sum()


inf


/home/floriansuri/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


66

In [50]:
cat_features = []
for col in train.columns:
    if train[col].dtype =='object':
        cat_features.append(col)
print(cat_features)

['FieldPosition', 'OffenseFormation', 'OffensePersonnel', 'DefensePersonnel', 'PlayDirection', 'TimeHandoff', 'TimeSnap', 'PlayerHeight', 'PlayerBirthDate', 'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather', 'WindDirection']


In [21]:
# encoding categorical values

from sklearn.preprocessing import LabelEncoder
new_obj=[]
for c in train.columns:
    if(train[c].dtype != int):
            try:
                train[c]=train[c].astype('float16')
            except:
                new_obj.append(c)
lbdic={}
for c in new_obj:
    lb=LabelEncoder()
    lb=lb.fit(train[c].values)
    lbdic[c]=lb
    train[c]=lb.transform(train[c].values)

TypeError: '<' not supported between instances of 'str' and 'float'

5.0                  26686
7.0                  26642
5                    26598
6.0                  22800
9.0                  19920
4                    19844
10.0                 19568
6                    18780
8.0                  18348
2.0                  17520
12.0                 17226
3                    15840
4                    14740
11.0                 14496
7                    12936
9                    11408
8                    11022
3                    11022
10                   10220
15.0                  7964
0                     7656
13                    7348
1                     6776
2                     6592
12                    6358
0.0                   6116
15                    5962
16.0                  5962
1                     5302
14.0                  4154
16                    3916
11                    2620
17.0                  2046
18.0                  1980
14                    1940
17                    1826
13 MPH                1804
2